# 通过Sharpe和Sortino来选股

In [6]:
await init_notebook()

In [13]:
APPROX_BDAYS_PER_YEAR = 252

def max_drawdown(returns: np.ndarray) -> Tuple:
    """计算最大资产回撤

    [代码引用](https://stackoverflow.com/a/22607546)
    Args:
        returns : 收益率（而不是资产净值）

    Returns:
        [Tuple]: mdd, start, send
    """
    if len(returns) < 1:
        raise ValueError("returns must have at least one values")

    equitity = np.nancumprod(returns + 1)
    i = np.nanargmax(np.fmax.accumulate(equitity) - equitity)
    if i == 0:
        return (0, 0, 0)

    j = np.nanargmax(equitity[:i])

    return (equitity[i] - equitity[j]) / equitity[j], i, j


def sharpe_ratio(
    returns: np.ndarray, rf: float = 0.0, annual_factor: int = APPROX_BDAYS_PER_YEAR
) -> float:
    """计算夏普比率(年化)

    平均超额收益（即收益减去无风险利率）除以标准差，即夏普比率。

    `rf`(risk-free利率)为年化无风险利率。

    关于年化因子，请参见[年化收益率][omicron.talib.metrics.annual_return]中的定义。

    Note:
        See [this article](https://towardsdatascience.com/sharpe-ratio-sorino-ratio-and-calmar-ratio-252b0cddc328) for more details.

    Args:
        returns: 回报率(一维数组).
        rf: risk free returns(年化). Defaults to 0.0.
        annual_factor: 年化因子，默认为`APPROX_BDAYS_PER_YEAR`。如果`returns`为日收益率，则`annual_factor`可使用默认值；否则，应该使用根据returns取得的周期，传入对应的年化因子。

    Raise:
        ValueError: 如果`returns`中的收益率少于1个有效值。
    Returns:
        夏普比率。
    """

    adj_returns = returns - rf / APPROX_BDAYS_PER_YEAR
    return (np.nanmean(adj_returns) * np.sqrt(annual_factor)) / np.nanstd(
        adj_returns, ddof=1
    )


def sortino_ratio(
    returns: np.ndarray, rf: float = 0.0, annual_factor: int = APPROX_BDAYS_PER_YEAR
) -> float:
    """计算Sortino比率

    Sortina比率是将收益与负收益的标准差进行比较。在这里，我们并非使用负收益的标准差，而是使用了一种称为[downside risk][omicron.talib.metrics.downside_risk]的方法，这种方法与[investopedia](https://www.investopedia.com/terms/s/sortinoratio.asp)、[Quantopian empyrical](https://github.com/quantopian/empyrical/blob/master/empyrical/stats.py)及[this article](https://towardsdatascience.com/sharpe-ratio-sorino-ratio-and-calmar-ratio-252b0cddc328)保持一致。

    关于年化因子，请参见[年化收益率][omicron.talib.metrics.annual_return]中的定义。

    Args:
        returns : 收益率
        rf ([]): 无风险利率，默认为0.0
        annual_factor: 年化因子，默认为252.

    Returns:
        [float]: Sortino比率
    """
    adj_returns = returns - rf / annual_factor

    annualized_dr = downside_risk(adj_returns, annual_factor)
    if annualized_dr == 0:
        return np.inf

    return np.nanmean(adj_returns) * annual_factor / annualized_dr


def downside_risk(adjust_returns: np.array, annual_factor: int = 252) -> float:
    """计算downside risk。downside risk在sortino ratio中使用。

    严格地说，sortino ratio中的downside risk应该是求其标准差，但[investopedia](https://www.investopedia.com/terms/d/downside-deviation.asp)中的算法如此，很多实现，比如[Quantopian](https://github.com/quantopian/empyrical/blob/master/empyrical/stats.py#downside_risk)，都是这样计算的，这里与他们保持一致。正因为这个原因，我们把这个函数定义为downside_risk，而不是downside_deviation。

    Examples:
        >>> returns = np.array([-0.02, 0.16, 0.31, 0.17, -0.11, 0.21, 0.26, -0.03, 0.38])
        >>> rf = 0.01
        >>> round(downside_risk(returns - rf, 1), 4)
        0.0433

    Args:
        adjust_returns: 已减去无风险利率的收益率。
        annual_factor: 年化因子

    Returns:
        downside risk
    """
    downside = np.clip(adjust_returns, -np.inf, 0)
    downside = np.nanmean(np.square(downside))
    return np.sqrt(downside * annual_factor)

In [65]:
def feature(code, name, bars, results, ft):
    close = bars["close"]
    returns = close[1:]/close[:-1] - 1
    
    if len(close) < 31:
        return
    
    row = []
    for n in [-30, -10, -5]:
        sharpe = sharpe_ratio(returns[n:], 0.03)
        sortino = sortino_ratio(returns[n:], 0.03)
        if np.isinf(sharpe) or np.isinf(sortino):
            return
        row.append(sharpe)
        row.append(sortino)
    
    results.append((code, name, *row ))

In [73]:
results = await scan(feature, 31, nstocks=-1, silent=True)
cols = ["code", "name", "sharpe30", "sortino30", "sharpe10", "sortino10", "sharpe5", "sortino5"]
df = pd.DataFrame(results, columns=cols)
df

2022-02-21
progress: 500/4033, results: 483, elapsed: 1, ETA: 7
progress: 1000/4033, results: 965, elapsed: 1, ETA: 3
progress: 1500/4033, results: 1439, elapsed: 2, ETA: 3
progress: 2000/4033, results: 1920, elapsed: 3, ETA: 3
progress: 2500/4033, results: 2409, elapsed: 4, ETA: 2
progress: 3000/4033, results: 2900, elapsed: 5, ETA: 1
progress: 3500/4033, results: 3387, elapsed: 6, ETA: 0
progress: 4000/4033, results: 3871, elapsed: 6, ETA: 0


,code,name,sharpe30,sortino30,sharpe10,sortino10,sharpe5,sortino5
0,000001.XSHE,平安银行,-0.012778,-0.019073,0.239324,0.337451,-1.630826,-2.350689
1,000002.XSHE,万科A,0.789740,1.135260,-1.780466,-2.369524,0.921003,1.815295
2,000004.XSHE,国华网安,1.933044,2.997625,1.471543,2.073259,-0.849394,-1.090237
3,000006.XSHE,深振业A,0.146715,0.203864,2.150495,4.551050,-0.645477,-1.125473
4,000009.XSHE,中国宝安,-0.793379,-1.087128,3.287319,7.165794,6.696791,27.106485
...,...,...,...,...,...,...,...,...
3899,605588.XSHG,冠石科技,-1.520531,-1.911995,3.127656,4.742583,9.988246,26.951896
3900,605589.XSHG,圣泉集团,-2.251984,-3.340201,2.608999,4.917299,-3.431531,-4.261596
3901,605598.XSHG,上海港湾,1.310362,2.175901,4.233654,7.191154,0.210222,0.350795
3902,605599.XSHG,菜百股份,-0.827440,-1.305249,-2.833022,-3.728762,-6.825624,-7.996588


In [83]:
def query(df, t1, t2):
    df = df.copy()
    #df = df[(df.sharpe30 < df.sharpe10) & (df.sharpe10 < df.sharpe5) & (df.sharpe30 >0)]
    
    df = df[(df.sortino30 < df.sortino10) & (df.sortino10 < df.sortino5) & (df.sortino30 > 0)]
    
    df = df[(df.sortino5<t2) & (df.sortino30>t1)]
    return df.sort_values("sortino5", ascending=False)

In [111]:
query(df, 1.8, 3.6)

,code,name,sharpe30,sortino30,sharpe10,sortino10,sharpe5,sortino5
2797,600606.XSHG,绿地控股,1.198047,1.832096,1.64281,2.886396,1.558086,3.048471


In [95]:
def exam(code, end, n):
    bars = jq.get_bars(code, n, '1d', end_dt = end, df = False)
    close = bars["close"]

    returns = close[1:]/close[:-1] - 1
    
    sharpe = sharpe_ratio(returns, 0.03)
    sortino = sortino_ratio(returns, 0.03)
    
    print(f"sharpe {round(sharpe, 1)}, sortino: {round(sortino, 1)}")

In [110]:
exam("002494.XSHE", "2021-12-29", 30)

sharpe 1.9, sortino: 3.1


In [102]:
bars = jq.get_bars("600283.XSHG", 30, '30m', end_dt = "2021-12-30", df = False)

In [104]:
dt = bars[0]["date"]